In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import random
import time
import matplotlib.pyplot as plt

In [2]:
data=np.load('data/new_data_train.npy')
X_test=np.load('data/new_data_test.npy')

data_all=np.copy(data)

data=list(data)
random.shuffle(data)
data=np.array(data)

num_data=data.shape[0]
num_test=X_test.shape[0]

print(num_data)
print(num_test)

ratio_train=0.8

data_train=data[:int(num_data*ratio_train),:]
data_val=data[int(num_data*ratio_train):,:]

X_train=data_train[:,:-1]
y_train=data_train[:,-1]
X_val=data_val[:,:-1]
y_val=data_val[:,-1]

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

260864
261477
(208691, 21)
(208691,)
(52173, 21)
(52173,)


In [3]:
num1=np.sum(y_train)
num0=y_train.shape[0]-num1
weight=num0/num1
print(weight)

data_train = xgb.DMatrix(X_train, label=y_train)
data_validation = xgb.DMatrix(X_val, label=y_val)
data_test = xgb.DMatrix(X_test)

data = xgb.DMatrix(data_all[:,:-1],label=data_all[:,-1])

param = {'objective':'binary:logistic', 'scale_pos_weight':weight, 'learning_rate':0.001, 'max_depth':8, 'eval_metric':'auc',
         'min_child_weight':1, 'subsample':1, 'nthread':4, 'n_estimators':100,
         'reg_alpha': 1, 'reg_lambda':1}

num_round = 5000

bst = xgb.train(param, data_train, num_boost_round=num_round, evals=[(data_validation,'val')], 
                verbose_eval =50, early_stopping_rounds=500)

15.4673715774
[0]	val-auc:0.601085
Will train until val-auc hasn't improved in 500 rounds.
[50]	val-auc:0.604215
[100]	val-auc:0.607356
[150]	val-auc:0.608527
[200]	val-auc:0.609553
[250]	val-auc:0.611671
[300]	val-auc:0.612211
[350]	val-auc:0.612503
[400]	val-auc:0.613215
[450]	val-auc:0.613639
[500]	val-auc:0.614134
[550]	val-auc:0.614414
[600]	val-auc:0.614635
[650]	val-auc:0.614725
[700]	val-auc:0.614798
[750]	val-auc:0.614871
[800]	val-auc:0.615003
[850]	val-auc:0.615102
[900]	val-auc:0.615271
[950]	val-auc:0.61538
[1000]	val-auc:0.615473
[1050]	val-auc:0.615544
[1100]	val-auc:0.615682
[1150]	val-auc:0.615838
[1200]	val-auc:0.61581
[1250]	val-auc:0.615752
[1300]	val-auc:0.615818
[1350]	val-auc:0.616002
[1400]	val-auc:0.616078
[1450]	val-auc:0.61607
[1500]	val-auc:0.616014
[1550]	val-auc:0.615954
[1600]	val-auc:0.615921


KeyboardInterrupt: 

In [ ]:
y_pred = bst.predict(data_validation)
print(y_pred)

In [ ]:
perform_his=[]
bounds=np.linspace(0.0,1.0,1001)

for bound in bounds:
    pre=y_pred>bound
    true_pos=np.sum(np.logical_and(pre==1,y_val==1))
    true_neg=np.sum(np.logical_and(pre==0,y_val==0))
    false_pos=np.sum(np.logical_and(pre==1,y_val==0))
    false_neg=np.sum(np.logical_and(pre==0,y_val==1))
    TPR=true_pos/(true_pos+false_neg)
    FPR=false_pos/(false_pos+true_neg)
    perform_his.append((FPR,TPR))
    plt.plot(FPR,TPR,'+')
    perform_his_s= sorted(perform_his, key=lambda d:d[0])
    
plt.show()

AUC=0
for i in range(1,len(perform_his_s)):
    pre_p=perform_his_s[i-1]
    cur_p=perform_his_s[i]
    AUC+=(cur_p[1]+pre_p[1])*(cur_p[0]-pre_p[0])/2
    
print('AUC =',AUC)

In [ ]:
ypred = bst.predict(data_test)
print(ypred)

In [ ]:
f_out=open('prediction.csv','w')
f_out.write('user_id,merchant_id,prob\n')
f_test=open('data_format1/test_format1.csv')
i=0
for line in f_test:
    l=line.strip()
    if l=='user_id,merchant_id,prob':
        continue
    [user,seller,_]=l.split(',')
    f_out.write(user+','+seller+','+str(ypred[i])+'\n')
    i+=1
f_out.close()